<a href="https://colab.research.google.com/github/David-Huson/CAP4601_Project2/blob/main/project02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAP4601-Intro to AI Project 2
## Image Classification using Convolutional Neural Networks

### Install and Import Dependencies

In [3]:
!pip install -q tensorflow tensorflow-datasets

In [ ]:
!pip install -U keras-tuner